## This notebook aims to fix the punctuation that was created by youtube in their transcripts. 

In [ ]:
import json
import pandas as pd
import numpy as np
from deepmultilingualpunctuation import PunctuationModel
import pickle
from tqdm import tqdm


In [ ]:
def get_data(path): 
        merged_df = pd.read_csv(path)
        strings = merged_df['String']
        str_lst = strings.values

        vocab = merged_df['Title'].values
        identifier = merged_df['identifier']
        identifier_vocab = pd.DataFrame({'ID': identifier, 'Vocab': vocab})
        identifier_vocab = identifier_vocab.set_index('Vocab')['ID'].to_dict()
        return merged_df, str_lst, vocab, identifier_vocab, identifier


merged_df, str_lst, vocab, identifier_vocab, identifier = get_data('data/merged_data_for_AI.csv')


In [ ]:
model = PunctuationModel()


In [ ]:
corrected = []
start_index = 0
end_index = len(merged_df["transcript"])
total_iterations = end_index- start_index
with tqdm(total=total_iterations) as pbar:
    for index, t in enumerate(merged_df["transcript"][start_index:end_index]):
        if index % 1000 == 0:
            # Save the corrected list to disk
            with open(f'corrected_transcripts_'+str(start_index) + 'to' + str(end_index) + '.pkl', 'wb') as f:
                pickle.dump(corrected, f)
        try:
            result = model.restore_punctuation(t)
            corrected.append(result)
        except Exception as e:
            print(e)
            corrected.append("ERROR: " + repr(e))
        pbar.update(1)

# Save the final corrected list to disk
with open(f'corrected_transcripts_'+str(start_index) + 'to' + str(end_index) + '_final.pkl', 'wb') as f:
                pickle.dump(corrected, f)
